In [43]:
import pickle
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import numpy
nltk.download('punkt')



def cluster_score(cluster):
    sig_words = len(cluster)
    total_words = cluster[-1] - cluster[0] + 1
    return sig_words ** 2 / total_words

def score_sentences(sentences, important_words, THRESH = 5):
    scores = []
    
    for sent in map(word_tokenize, sentences):
        word_idx = []
        
        for word in important_words:
            if word in sent:
                word_idx.append(sent.index(word))
        word_idx.sort()
        if len(word_idx) > 0:
            clusters = []
            current_cluster = [word_idx[0]]
            for idx in word_idx[1:]:
                if idx - word_idx[-1] < THRESH:
                    current_cluster.append(idx)
                else:
                    clusters.append(current_cluster)
                    current_cluster = [idx]
            clusters.append(current_cluster)
            scores.append(max(map(cluster_score, clusters)))
        else:
            scores.append(0)
    return scores

def summarize(sentences, important_words, CTHRESH = 5, NUM_SENTENCES = 5):
    scores = score_sentences(sentences, important_words, CTHRESH)
    avg = numpy.mean(scores)
    std_dev = numpy.std(scores)
    score_threshold = avg + 0.5 * std_dev
    mean_scored = [t[0] for t in enumerate(scores) if t[1] > score_threshold]
    
    sorted_scores = sorted(enumerate(scores), key=lambda x: x[1], reverse = True)[:NUM_SENTENCES]
    sorted_indexes = sorted([s[0] for s in sorted_scores])
    
    return (' '.join([sentences[i] for i in sorted_indexes]))

def sentence_cleaner(sentences):
    listsents = []
    for a in range(len(sentences)):
        for sent in sentences[a]:
            newsents = re.sub('[-_\/]', '', sent)
            listsents.append(newsents)
    return listsents

def word_cleaner(sentences):
    newList = []
    for a in range(len(sentences)):
        for sent in sentences[a]:
            newsent = re.sub('[^a-zA-Z]', ' ', sent)
            newList.append(newsent)
    tokens = [word_tokenize(s) for s in newList]
    article_words = []
    for toke in tokens:
        article_words += toke
    return article_words

def most_common_words(words, num_sents):
    stopWords = set(stopwords.words("english"))
    for punct in "_-,.'?:;'""":
        stopWords.add(punct)
    freqDist = nltk.FreqDist(words)
    sorted_words = sorted(freqDist.items(), key=lambda x: x[1], reverse=True)
    common_words = [word[0] for word in sorted_words
                    if word[0] not in stopWords][:num_sents]
    return common_words

def article_sentences(request):
    soup = BeautifulSoup(request.text, 'html.parser')
    paras = soup.findAll('p', string=True)
    paragraphs = []
    for para in paras:
        paragraphs.append(para.text)
    sentences = []
    sentences = [sent_tokenize(sent) for sent in paragraphs]
    return sentences

def article_title(request):
    souper = BeautifulSoup(request.text, 'html.parser')
    souptitler = souper.find('title')
    titler = souptitler.text
    sentTitler = sent_tokenize(titler)
    phraseList = []
    for sent in sentTitler:
        phrasesent = re.sub('[^a-zA-Z]', ' ', sent)
        phraseList.append(phrasesent)
    tokeny = [word_tokenize(s) for s in phraseList]
    filtered_words = [word for word in tokeny if word not in stopwords.words('english')]
    return filtered_words
    
#https://www.msn.com/en-us/news/world/putin-russia-knows-mueller-probe-gave-birth-to-a-mouse/ar-BBVLtjG?ocid=spartanntp
#https://www.msn.com/en-us/health/health-news/a-new-cancer-vaccine-uses-the-immune-system-to-take-out-tumors/ar-BBVLDdK?ocid=spartanntp
one = requests.get('https://www.msn.com/en-us/news/world/putin-russia-knows-mueller-probe-gave-birth-to-a-mouse/ar-BBVLtjG?ocid=spartanntp')
two = requests.get('https://www.msn.com/en-us/money/companies/worlds-2-biggest-beer-brewers-in-battle-over-keg-technology/ar-BBVZLRj?ocid=spartanntp')



oneSentences = article_sentences(one)

twoSentences = article_sentences(two)


with open('sentences.pkl', 'wb') as f:
    pickle.dump(rsentences, f)
with open('sentences.pkl', 'rb') as f:
    pickled_sents = pickle.load(f)


oneCleanSents = sentence_cleaner(oneSentences)

twoCleanSents = sentence_cleaner(twoSentences)



oneClean_words = word_cleaner(oneSentences)

twoClean_words = word_cleaner(twoSentences)


num_sents = 5
one_common_words = most_common_words(oneClean_words, num_sents)
print(common_words)


one_common_word_summary = summarize(oneCleanSents, one_common_words, 5, 5)
print(one_common_word_summary)

print('    ')


one_titlePhrase = article_title(one)

print(one_titlePhrase)

one_title_summary = summarize(oneCleanSents, one_titlePhrase, 5, 5)
print(one_title_summary)

print('    ')
two_common_words = most_common_words(twoClean_words, num_sents)
print(two_common_words)
two_common_word_summary = summarize(twoCleanSents, two_common_words, 5, 5)
print(two_common_word_summary)

print('    ')

two_Title = article_title(two)
print(two_Title)


two_title_summary = summarize(twoCleanSents, two_Title, 5, 5)
print(two_title_summary)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jzech\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jzech\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jzech\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Trump', 'Putin', 'Russian', 'Mueller', 'Russia']
ST. PETERSBURG, Russia  — Russian President Vladimir Putin on Tuesday mocked U.S. special counsel Robert Mueller's investigation of Kremlin interference in the 2016 presidential election, saying "a mountain gave birth to a mouse." However, Mueller uncovered evidence of a Kremlin operation to interfere with the 2016 vote. He charged 12 Russian military intelligence officers with breaking into Democratic Party computers and the email accounts of officials with Hillary Clinton's campaign. Another indictment detailed Russia's use of phony social media accounts to spread divisive rhetoric and to undermine the U.S. political system. Putin alleged Trump's political foes now were "searching for new pretexts" to undermine him.
    
[['Putin', 'Russia', 'knows', 'Mueller', 'probe', 'gave', 'birth', 'to', 'a', 'mouse']]
ST. PETERSBURG, Russia  — Russian President Vladimir Putin on Tuesday mocked U.S. special counsel Robert Mueller's investigation